## creating crossproduct of all possible dates and municipalities

In [ ]:
import pandas as pd
import itertools

In [ ]:
df_damages = pd.read_excel('../Data/USDB_1972_2023_ohneSchadenszahlen_ohneBeschriebe.xlsx')
df_damages.sort_values(by='Datum', inplace=True)
oldest_date = df_damages['Datum'].min()
newest_date = df_damages['Datum'].max()
oldest_date, newest_date

In [ ]:
init_len_damages = len(df_damages)
df_damages

In [ ]:
df_damages_municipality_date_hasDamage = df_damages[['Gemeinde', 'Datum', 'Schadensausmass, gering [0.01-0.4], mittel [0.4-2], gross/katastrophal[>2] oder Todesfall [Mio. CHF]', 'Hauptprozess']]
df_damages_municipality_date_hasDamage['damage'] = '1'
df_damages_municipality_date_hasDamage.drop_duplicates(inplace=True)
df_damages_municipality_date_hasDamage

In [ ]:
df_dates = pd.date_range(oldest_date, newest_date)
df_dates = pd.DataFrame({'Date': df_dates})
df_dates

In [ ]:
df_munuicipalities = pd.read_csv('../Data/municipalities/municipality_coordinates.csv')

In [ ]:
all_combinations = df_munuicipalities.merge(df_dates, how='cross')

In [ ]:
all_combinations_len = len(all_combinations)
all_combinations_len

In [ ]:
all_combinations

In [ ]:
df_damages_municipality_date_hasDamage.rename(columns={'Gemeinde': 'Municipality', 'Datum': 'Date'}, inplace=True)

In [ ]:
df_damages_municipality_date_hasDamage.rename(columns={'Schadensausmass, gering [0.01-0.4], mittel [0.4-2], gross/katastrophal[>2] oder Todesfall [Mio. CHF]': 'extent_of_damage', 'Hauptprozess': 'main_process'}, inplace=True)

In [ ]:
df_damages_municipality_date_hasDamage

In [ ]:
#remap the extend of damage column:
damage_mapping = {
    'gering': 1,
    'mittel': 2,
    'gross/katastrophal': 3
}

df_damages_municipality_date_hasDamage['extent_of_damage'] = df_damages_municipality_date_hasDamage['extent_of_damage'].map(damage_mapping)
df_damages_municipality_date_hasDamage

In [ ]:
df_damages_municipality_date_hasDamage['main_process'].unique()

In [ ]:
#remap the main process
main_process_mapping = {
    'Sturz': 1,
    'Rutschung': 2,
    'Wasser/Murgang': 3
}

df_damages_municipality_date_hasDamage['main_process'] = df_damages_municipality_date_hasDamage['main_process'].map(main_process_mapping)
df_damages_municipality_date_hasDamage

In [ ]:
df_damages_with_municipality_date_comb = all_combinations.merge(
    df_damages_municipality_date_hasDamage,
    on=['Municipality', "Date"],
    how="left"
)
df_damages_with_municipality_date_comb

In [ ]:
df_damages_with_municipality_date_comb['damage'] = df_damages_with_municipality_date_comb['damage'].fillna(0)
df_damages_with_municipality_date_comb['extent_of_damage'] = df_damages_with_municipality_date_comb['extent_of_damage'].fillna(0)
df_damages_with_municipality_date_comb['main_process'] = df_damages_with_municipality_date_comb['main_process'].fillna(0)

In [ ]:
df_damages_with_municipality_date_comb


In [ ]:
df_damages_with_municipality_date_comb.to_csv('../Data/all_data.csv')